In [1]:
import argparse
import os
import sys
import tabulate
import time
import torch
import torch.nn.functional as F

import curves
import data
import models
import utils
import pickle

import copy
import torchvision

In [2]:
architecture = getattr(models, "VGG16")

In [3]:
architecture

models.vgg.VGG16

In [4]:
model = architecture.base(num_classes=10, **architecture.kwargs)

In [9]:
loaders, num_classes = data.loaders(
    "CIFAR10",
    "data",
    128,
    1,
    "VGG",
    False
)

Files already downloaded and verified
Using train (45000) + validation (5000)
Files already downloaded and verified


In [10]:
for X, y in loaders['test']:
    break

In [11]:
# from matplotlib import pyplot as plt
# %matplotlib inline

# def show_images(x, scale=15, line_width=10):
#     plt.figure(figsize=(scale, scale / line_width * (x.shape[0] // line_width + 1)))
#     x = x.view(-1, 1, 28, 28)
#     mtx = torchvision.utils.make_grid(x, nrow=line_width, pad_value=1)
#     plt.imshow(mtx.permute([1, 2, 0]).numpy(), cmap='Greys_r', vmin=0, vmax=1, interpolation='lanczos')
#     plt.axis('off')

# show_images(X[:10])

In [12]:
y_pred = torch.argmax(model(X), dim=-1)

In [13]:
y

tensor([ 7,  1,  4,  1,  0,  2,  2,  5,  9,  6,  0,  1,  3,  1,
         1,  2,  1,  8,  5,  6,  9,  2,  4,  2,  8,  8,  3,  7,
         1,  7,  1,  7,  5,  9,  7,  0,  7,  8,  3,  6,  1,  2,
         2,  5,  9,  8,  4,  8,  9,  4,  2,  5,  9,  3,  7,  9,
         9,  3,  7,  9,  2,  7,  8,  9,  0,  7,  3,  0,  1,  7,
         1,  4,  6,  8,  2,  3,  4,  1,  6,  2,  6,  0,  3,  5,
         1,  0,  9,  7,  0,  0,  9,  3,  4,  9,  8,  8,  0,  1,
         6,  1,  4,  9,  0,  7,  8,  1,  0,  9,  5,  2,  4,  4,
         7,  0,  4,  8,  9,  3,  1,  2,  2,  4,  4,  0,  4,  5,
         1,  7])

In [14]:
y_pred

tensor([ 8,  1,  8,  8,  1,  8,  1,  8,  8,  1,  8,  1,  8,  8,
         8,  1,  8,  1,  8,  1,  8,  8,  1,  1,  9,  8,  9,  8,
         1,  1,  1,  8,  1,  8,  9,  1,  1,  8,  1,  8,  1,  1,
         1,  1,  1,  8,  8,  8,  8,  1,  8,  1,  1,  1,  1,  8,
         8,  8,  1,  8,  8,  1,  8,  8,  1,  1,  9,  1,  1,  8,
         8,  1,  8,  1,  8,  1,  8,  8,  8,  8,  8,  8,  1,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  1,  1,  8,  1,  8,  8,
         8,  8,  1,  1,  8,  8,  1,  8,  1,  8,  1,  1,  1,  8,
         8,  8,  1,  8,  8,  8,  8,  8,  1,  1,  8,  8,  8,  8,
         1,  8])

In [15]:
model

VGGBase(
  (layer_blocks): ModuleList(
    (0): ModuleList(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): ModuleList(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): ModuleList(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (3): ModuleList(
      (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (4): ModuleList(
      (0): Conv2d(512, 512, kernel_size=(3, 3), 

## Rescale

In [115]:
model = architecture.base(num_classes=10, **architecture.kwargs)
model_base = architecture.base(num_classes=10, **architecture.kwargs)

# model.load_state_dict(torch.load('curves/curve1/checkpoint-100.pt')['model_state'])
# model_base.load_state_dict(torch.load('curves/curve1/checkpoint-100.pt')['model_state'])

for p1, p2 in zip(model_base.parameters(), model.parameters()):
    p1.data.copy_(p2)

model.train(False);
model_base.train(False);

In [116]:
y_pred1 = torch.argmax(model(X), dim=-1)
y_pred2 = torch.argmax(model_base(X), dim=-1)

(~(y_pred1 == y_pred2)).sum()

tensor(0)

In [117]:
def rescale(l, scale):
    list(model.modules())[l].weight=torch.nn.Parameter(list(model.modules())[l].weight*scale)
    list(model.modules())[l].bias=torch.nn.Parameter(list(model.modules())[l].bias*scale)

In [118]:
list(model.modules())[18]

Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [119]:
rescale(-3, 10)

In [120]:
rescale(-6, 0.1)

In [121]:
y_pred_r = torch.argmax(model(X), dim=-1)

In [123]:
sum(~(y_pred2==y_pred_r))

tensor(0, dtype=torch.uint8)

In [124]:
for X, y in loaders['test']:
    y_pred1 = torch.argmax(model(X), dim=-1)
    y_pred2 = torch.argmax(model_base(X), dim=-1)
    print((~(y_pred1 == y_pred2)).sum())

tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(1)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(1)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)


## Node

In [125]:
def change_node(l1, l2, i, j):
    #   matrix
    c = copy.deepcopy(torch.nn.Parameter(list(model.modules())[l1].weight[j]))
    list(model.modules())[l1].weight[j] = list(model.modules())[l1].weight[i]
    list(model.modules())[l1].weight[i] = c

    #     bias
    c = copy.deepcopy(torch.nn.Parameter(list(model.modules())[l1].bias[j]))
    list(model.modules())[l1].bias[j] = list(model.modules())[l1].bias[i]
    list(model.modules())[l1].bias[i] = c

    c = copy.deepcopy(torch.nn.Parameter(list(model.modules())[l2].weight.transpose(0, 1)[j]))
    list(model.modules())[l2].weight.transpose(0, 1)[j] = list(model.modules())[l2].weight.transpose(0, 1)[i]
    list(model.modules())[l2].weight.transpose(0, 1)[i] = c

In [126]:
model = architecture.base(num_classes=10, **architecture.kwargs)
model_base = architecture.base(num_classes=10, **architecture.kwargs)

# model.load_state_dict(torch.load('curves/curve1/checkpoint-100.pt')['model_state'])
# model_base.load_state_dict(torch.load('curves/curve1/checkpoint-100.pt')['model_state'])

for p1, p2 in zip(model_base.parameters(), model.parameters()):
    p1.data.copy_(p2)
    
model.train(False);
model_base.train(False);

In [127]:
y_pred1 = torch.argmax(model(X), dim=-1)
y_pred2 = torch.argmax(model_base(X), dim=-1)

(~(y_pred1 == y_pred2)).sum()

tensor(0)

In [128]:
i = 10
for ind, (p1, p2) in enumerate(zip(model.parameters(), model_base.parameters())):
    div = (list(model.parameters())[0]==list(model_base.parameters())[0]).sum().numpy()/np.prod(list(model.parameters())[0].shape)
    print(ind, div, p1.shape)

0 1.0 torch.Size([64, 3, 3, 3])
1 1.0 torch.Size([64])
2 1.0 torch.Size([64, 64, 3, 3])
3 1.0 torch.Size([64])
4 1.0 torch.Size([128, 64, 3, 3])
5 1.0 torch.Size([128])
6 1.0 torch.Size([128, 128, 3, 3])
7 1.0 torch.Size([128])
8 1.0 torch.Size([256, 128, 3, 3])
9 1.0 torch.Size([256])
10 1.0 torch.Size([256, 256, 3, 3])
11 1.0 torch.Size([256])
12 1.0 torch.Size([256, 256, 3, 3])
13 1.0 torch.Size([256])
14 1.0 torch.Size([512, 256, 3, 3])
15 1.0 torch.Size([512])
16 1.0 torch.Size([512, 512, 3, 3])
17 1.0 torch.Size([512])
18 1.0 torch.Size([512, 512, 3, 3])
19 1.0 torch.Size([512])
20 1.0 torch.Size([512, 512, 3, 3])
21 1.0 torch.Size([512])
22 1.0 torch.Size([512, 512, 3, 3])
23 1.0 torch.Size([512])
24 1.0 torch.Size([512, 512, 3, 3])
25 1.0 torch.Size([512])
26 1.0 torch.Size([512, 512])
27 1.0 torch.Size([512])
28 1.0 torch.Size([512, 512])
29 1.0 torch.Size([512])
30 1.0 torch.Size([10, 512])
31 1.0 torch.Size([10])


In [82]:
# list(model.modules())[0]

In [129]:
list(model.modules())[18]

Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [130]:
for i in range(256):
    change_node(17, 18, i, i+256)

In [131]:
for X, y in loaders['test']:
    y_pred1 = torch.argmax(model(X), dim=-1)
    y_pred2 = torch.argmax(model_base(X), dim=-1)
    print((~(y_pred1 == y_pred2)).sum())

tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)


In [111]:
y_pred_n = torch.argmax(model(X), dim=-1)

In [112]:
y_pred_n

tensor([ 2,  2,  6,  2,  3,  2,  6,  2,  2,  6,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  6,  2,  6,  6,  6,  3,  2,  6,  3,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  6,  2,
         3,  2,  2,  2,  2,  2,  2,  3,  6,  2,  2,  2,  2,  2,
         3,  2,  2,  3,  2,  3,  2,  2,  2,  3,  3,  3,  2,  3,
         3,  6,  6,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         3,  2,  2,  2,  2,  2,  2,  2,  6,  2,  2,  2,  2,  2,
         2,  2,  2,  3,  3,  3,  2,  3,  2,  2,  3,  2,  6,  2,
         2,  2,  6,  6,  2,  2,  2,  3,  2,  2,  2,  3,  2,  2,
         2,  2])

In [114]:
print("eq ", (~(y_pred_n==y_pred1)).sum())

eq  tensor(0)


## Saving

In [71]:
#  torch.load("curve/checkpoint-50.pt")

In [74]:
for path, k in [("Aaa", 0), ("Bbb", 4 - 1)]:
    print('p ', path)
    print('k', k)
    

p  Aaa
k 0
p  Bbb
k 3


In [77]:
a = list(range(10))
a[::3]

[0, 3, 6, 9]

In [70]:
architecture = getattr(models, "VGG16")
curve = getattr(curves, 'PolyChain')

In [71]:
curve

curves.PolyChain

In [72]:
architecture.curve

models.vgg.VGGCurve

In [73]:
architecture.kwargs

{'batch_norm': False, 'depth': 16}

In [74]:
model = curves.CurveNet(
        10,
        curve,
        architecture.curve,
        3,
        True,
        True,
        architecture_kwargs=architecture.kwargs,
    )

In [75]:
model

CurveNet(
  (coeff_layer): PolyChain()
  (net): VGGCurve(
    (layer_blocks): ModuleList(
      (0): ModuleList(
        (0): Conv2d()
        (1): Conv2d()
      )
      (1): ModuleList(
        (0): Conv2d()
        (1): Conv2d()
      )
      (2): ModuleList(
        (0): Conv2d()
        (1): Conv2d()
        (2): Conv2d()
      )
      (3): ModuleList(
        (0): Conv2d()
        (1): Conv2d()
        (2): Conv2d()
      )
      (4): ModuleList(
        (0): Conv2d()
        (1): Conv2d()
        (2): Conv2d()
      )
    )
    (activation_blocks): ModuleList(
      (0): ModuleList(
        (0): ReLU(inplace)
        (1): ReLU(inplace)
      )
      (1): ModuleList(
        (0): ReLU(inplace)
        (1): ReLU(inplace)
      )
      (2): ModuleList(
        (0): ReLU(inplace)
        (1): ReLU(inplace)
        (2): ReLU(inplace)
      )
      (3): ModuleList(
        (0): ReLU(inplace)
        (1): ReLU(inplace)
        (2): ReLU(inplace)
      )
      (4): ModuleList(
        (

In [76]:
curve_parameters = list(model.net.parameters())

In [87]:
len(curve_parameters)

96

In [88]:
for i, p in enumerate(curve_parameters):
    print(i, p.shape)

0 torch.Size([64, 3, 3, 3])
1 torch.Size([64, 3, 3, 3])
2 torch.Size([64, 3, 3, 3])
3 torch.Size([64])
4 torch.Size([64])
5 torch.Size([64])
6 torch.Size([64, 64, 3, 3])
7 torch.Size([64, 64, 3, 3])
8 torch.Size([64, 64, 3, 3])
9 torch.Size([64])
10 torch.Size([64])
11 torch.Size([64])
12 torch.Size([128, 64, 3, 3])
13 torch.Size([128, 64, 3, 3])
14 torch.Size([128, 64, 3, 3])
15 torch.Size([128])
16 torch.Size([128])
17 torch.Size([128])
18 torch.Size([128, 128, 3, 3])
19 torch.Size([128, 128, 3, 3])
20 torch.Size([128, 128, 3, 3])
21 torch.Size([128])
22 torch.Size([128])
23 torch.Size([128])
24 torch.Size([256, 128, 3, 3])
25 torch.Size([256, 128, 3, 3])
26 torch.Size([256, 128, 3, 3])
27 torch.Size([256])
28 torch.Size([256])
29 torch.Size([256])
30 torch.Size([256, 256, 3, 3])
31 torch.Size([256, 256, 3, 3])
32 torch.Size([256, 256, 3, 3])
33 torch.Size([256])
34 torch.Size([256])
35 torch.Size([256])
36 torch.Size([256, 256, 3, 3])
37 torch.Size([256, 256, 3, 3])
38 torch.Size([2

In [81]:
model.net.parameters()

<generator object Module.parameters at 0x151b1c9a40>

In [89]:
a = np.array([1,4,5,7])

In [95]:
a[(a>3)*(a<7)]

array([4, 5])

In [94]:
pred = model(X)

In [95]:
pred.shape

torch.Size([128, 10])

In [96]:
from torch.nn import Module

In [98]:
m = Module()

In [105]:
m.register_buffer('range', torch.arange(0, float(3)))

In [106]:
m.range

tensor([ 0.,  1.,  2.])

In [122]:
t_n = 2/3 * (3 - 1)
torch.max(m.range.new([0.0]), 1.0 - torch.abs(t_n - m.range))

tensor([ 0.0000,  0.6667,  0.3333])